In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace,lit, desc, lower, explode
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import json

In [2]:
spark = SparkSession.builder \
    .appName("Recomendation System") \
    .getOrCreate()

In [3]:
df = spark.read.parquet('hdfs:///eaplayer_data/male_players_version_23.parquet')

In [4]:
club_positions_df = df.select("club_position")

In [5]:
club_positions_rdd = club_positions_df.rdd

In [6]:
positions_rdd = club_positions_rdd.map(lambda row: row[0])

In [7]:
unique_positions_rdd = positions_rdd.distinct()

In [8]:
col_positions = unique_positions_rdd.collect()

### recomendation system 1st try (no criteria)
- No criteria what so ever. The code Iterates through positions and selects the best available player, one by one.

In [9]:
""" recommended_team = []
selected_player_ids = set()  

for position in col_positions:
    position_players = df.filter(col('club_position').isNotNull() & (col('club_position') != ''))
    
    selected_players = position_players.select(
        "player_id", 
        "short_name", 
        "overall", 
        lit(position).alias("position")
    ).filter(~col("player_id").isin(selected_player_ids))  

    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()
    
    if best_player:
        player = best_player[0]
        recommended_team.append(player)
        selected_player_ids.add(player['player_id'])  

print("\nRecommended Team:")
for player in recommended_team:
    print(f"Position: {player['position']}, Player ID: {player['player_id']}, Name: {player['short_name']}, Overall: {player['overall']}")
 """

' recommended_team = []\nselected_player_ids = set()  \n\nfor position in col_positions:\n    position_players = df.filter(col(\'club_position\').isNotNull() & (col(\'club_position\') != \'\'))\n    \n    selected_players = position_players.select(\n        "player_id", \n        "short_name", \n        "overall", \n        lit(position).alias("position")\n    ).filter(~col("player_id").isin(selected_player_ids))  \n\n    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()\n    \n    if best_player:\n        player = best_player[0]\n        recommended_team.append(player)\n        selected_player_ids.add(player[\'player_id\'])  \n\nprint("\nRecommended Team:")\nfor player in recommended_team:\n    print(f"Position: {player[\'position\']}, Player ID: {player[\'player_id\']}, Name: {player[\'short_name\']}, Overall: {player[\'overall\']}")\n '

- Above we can see that good players like Ronaldo and Messi do show up.
- But we are not sure If thats the best team considering their age.

### Below updating the previous code, adding age and overall score criteria

In [10]:
""" recommended_team = []
selected_player_ids = set() 

for position in col_positions:
    position_players = df.filter(
        (col('club_position') == position) & 
        (col("age") < 30) &  
        (col("overall") > 80) 
    )

    selected_players = position_players.select(
        "player_id", 
        "short_name", 
        "overall", 
        "age",
        lit(position).alias("position")
    ).filter(~col("player_id").isin(selected_player_ids))

    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()
    
   
    if best_player:
        player = best_player[0]
        recommended_team.append(player)
        selected_player_ids.add(player['player_id']) 


print("\nRecommended Team:")
for player in recommended_team:
    print(f"Position: {player['position']}, Player ID: {player['player_id']}, Name: {player['short_name']}, Overall: {player['overall']}, Age: {player['age']}") """

' recommended_team = []\nselected_player_ids = set() \n\nfor position in col_positions:\n    position_players = df.filter(\n        (col(\'club_position\') == position) & \n        (col("age") < 30) &  \n        (col("overall") > 80) \n    )\n\n    selected_players = position_players.select(\n        "player_id", \n        "short_name", \n        "overall", \n        "age",\n        lit(position).alias("position")\n    ).filter(~col("player_id").isin(selected_player_ids))\n\n    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()\n    \n   \n    if best_player:\n        player = best_player[0]\n        recommended_team.append(player)\n        selected_player_ids.add(player[\'player_id\']) \n\n\nprint("\nRecommended Team:")\nfor player in recommended_team:\n    print(f"Position: {player[\'position\']}, Player ID: {player[\'player_id\']}, Name: {player[\'short_name\']}, Overall: {player[\'overall\']}, Age: {player[\'age\']}") '

- here we can see that famous old players have been removed. 
- but young and upcoming players like Mbappe Come up
- issue with this one is that we are applying a single criteria to all players

### Different criteria for every spot

In [11]:
position_criteria = {
    "cam": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75),
    "ls": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75) & (col("skill_dribbling") > 88) ,
    "cb": (col("age") < 30) & (col("overall") > 80),
    "lb": (col("potential") > 88) & (col("age") < 30) & (col("weight_kg") < 80) ,
    "rcm": (col("potential") > 85) & (col("age") < 30) & (col("power_long_shots") > 70)  & (col("skill_ball_control") > 85) &(col("defending") > 77),
    "rwb":  (col("potential") > 85) & (col("age") < 30),
    "res": (col("age") < 30) & (col("overall") > 80),
    "rdm": (col("potential") > 82) & (col("age") < 30) & (col("defending_standing_tackle")>80 ) & (col("passing") > 85),
    "ldm": (col("potential") > 82) & (col("age") < 30) & (col("defending_standing_tackle")>80 ) & (col("passing") > 80),
    "st": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75)  &(col("skill_dribbling") > 85) & (col("attacking_finishing") > 78) & (col("movement_acceleration") > 95),
    "none": (col("age") < 30) & (col("overall") > 80),
    "sub": (col("age") < 30) & (col("overall") > 80),
    "rb": (col("potential") > 82) & (col("age") < 30) & (col("defending_standing_tackle")>80 ) & (col("passing") > 80) ,
    "lcb":  (col("potential") > 82) & (col("age") < 30) & (col("defending_standing_tackle")>75 ) & (col("passing") > 75),
    "rcb": (col("potential") > 85) & (col("age") < 30) &(col("defending_sliding_tackle") > 80) & (col("passing") > 67),
    "lcm": (col("potential") > 85) & (col("age") < 30) & (col("power_long_shots") > 70)  & (col("skill_ball_control") > 70) &(col("defending") > 70) & (col('skill_dribbling') > 80),
    "lm": (col("age") < 30) & (col("overall") > 80),
    "cdm": (col("potential") > 85) & (col("age") < 30) & (col("passing") > 70) &(col("defending") > 70) & (col('skill_dribbling') > 70)  & (col("height_cm") > 190) &(col("weight_kg") > 80) ,
    "lw": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75)  &(col("skill_dribbling") > 85) & (col("attacking_finishing") > 78),
    "lwb":  (col("potential") > 85) & (col("age") < 30) &(col("skill_dribbling") > 70),
    "rm": (col("potential") > 85) & (col("age") < 30) & (col("skill_ball_control") > 70) &(col("defending") > 60) & (col('skill_dribbling') > 60),
    "cm":  (col("potential") > 85) & (col("age") < 30) & (col("power_long_shots") > 70)  & (col("skill_ball_control") > 70) & (col('skill_dribbling') > 80) &(col("defending_marking_awareness") > 70),
    "rs": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75) & (col("skill_dribbling") > 80),
    "lf": (col("potential") > 85) & (col("age") < 30) & (col("skill_ball_control") > 70) &(col("defending") > 60) & (col('skill_dribbling') > 60),
    "rw": (col("potential") > 85) & (col("age") < 30) & (col("shooting") > 75)  &(col("skill_dribbling") > 85) & (col("attacking_finishing") > 78),
    "cf": (col("age") < 30) & (col("overall") > 80),
    "lam": (col("potential") > 85) & (col("age") < 30) & (col("power_long_shots") > 70)  & (col("skill_ball_control") > 70) & (col('skill_dribbling') > 80) &(col("attacking_finishing")>75),
    "ram": (col("age") < 30) & (col("overall") > 80),
    "rf": (col("age") < 30) & (col("overall") > 80),
}


In [12]:
''' recommended_team = []
selected_player_ids = set()

available_players = df


for position in col_positions:
    print(f"\nEvaluating position: {position}")
    filter_condition = position_criteria.get(position)
    position_players = available_players.filter(
        (col('club_position').contains(position)) & 
        filter_condition 
    )

    num_available = position_players.count()
    print(f"Number of players available for {position}: {num_available}")

    selected_players = position_players.select(
        "player_id", 
        "short_name", 
        "overall", 
        "age",
        lit(position).alias("position")
    ).filter(~col("player_id").isin(selected_player_ids))

 
    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()
    
    if best_player:
        player = best_player[0]
        recommended_team.append(player)
        selected_player_ids.add(player['player_id']) 
        print(f"Selected Player for {position}: {player['short_name']}, Overall: {player['overall']}, Age: {player['age']}")
        available_players = available_players.filter(~(col("player_id") == player['player_id']))
    else:
        print(f"No suitable player found for position: {position}")

print("\nRecommended Team:")
for player in recommended_team:
    print(f"Position: {player['position']}, Player ID: {player['player_id']}, Name: {player['short_name']}, Overall: {player['overall']}, Age: {player['age']}")
'''

' recommended_team = []\nselected_player_ids = set()\n\navailable_players = df\n\n\nfor position in col_positions:\n    print(f"\nEvaluating position: {position}")\n    filter_condition = position_criteria.get(position)\n    position_players = available_players.filter(\n        (col(\'club_position\').contains(position)) & \n        filter_condition \n    )\n\n    num_available = position_players.count()\n    print(f"Number of players available for {position}: {num_available}")\n\n    selected_players = position_players.select(\n        "player_id", \n        "short_name", \n        "overall", \n        "age",\n        lit(position).alias("position")\n    ).filter(~col("player_id").isin(selected_player_ids))\n\n \n    best_player = selected_players.orderBy(desc("overall")).limit(1).collect()\n    \n    if best_player:\n        player = best_player[0]\n        recommended_team.append(player)\n        selected_player_ids.add(player[\'player_id\']) \n        print(f"Selected Player for {p

### this one works the best!!!!!!!

In [13]:
recommended_team = []
selected_player_ids = set()


for position in col_positions:
    criteria = position_criteria.get(position)
    position_candidates = (
        df
        .filter((col("club_position") == position) & criteria & (~col("player_id").isin(selected_player_ids)))
        .orderBy(desc("overall"))
        .limit(1)
    ).collect()
    
    if position_candidates:
        best_player = position_candidates[0]
        recommended_team.append({
            "position": position,
            "player_id": best_player["player_id"],
            "short_name": best_player["short_name"],
            "overall": best_player["overall"],
            "age": best_player["age"]
        })
        selected_player_ids.add(best_player["player_id"])

print("\nRecommended Team:")
for player in recommended_team:
    print(f"Position: {player['position']}, Player ID: {player['player_id']}, "
          f"Name: {player['short_name']}, Overall: {player['overall']}, Age: {player['age']}")



Recommended Team:
Position: lcm, Player ID: 225193, Name: Merino, Overall: 83, Age: 26
Position: sub, Player ID: 218667, Name: Bernardo Silva, Overall: 88, Age: 27
Position: lcb, Player ID: 239301, Name: L. Martínez, Overall: 82, Age: 24
Position: cdm, Player ID: 231866, Name: Rodri, Overall: 87, Age: 26
Position: rcb, Player ID: 205452, Name: A. Rüdiger, Overall: 87, Age: 29
Position: rcm, Player ID: 223848, Name: S. Milinković-Savić, Overall: 86, Age: 27
Position: lb, Player ID: 234396, Name: A. Davies, Overall: 84, Age: 21
Position: res, Player ID: 210315, Name: Raúl De Tomás, Overall: 83, Age: 27
Position: rw, Player ID: 237692, Name: P. Foden, Overall: 85, Age: 22
Position: cf, Player ID: 216547, Name: Rafa, Overall: 82, Age: 29
Position: rb, Player ID: 231281, Name: T. Alexander-Arnold, Overall: 87, Age: 23
Position: lw, Player ID: 200104, Name: H. Son, Overall: 89, Age: 29
Position: st, Player ID: 231747, Name: K. Mbappé, Overall: 91, Age: 23
Position: ls, Player ID: 232411, Na

In [ ]:
import json
output_file = "recommended_team.json"
with open(output_file, "w") as json_file:
    json.dump(recommended_team, json_file, indent=4)
print(f"Recommended team saved to {output_file}")


Recommended team saved to recommended_team.json


##### following codes where just used to check how many particular players we have if we apply some rules.

In [ ]:
""" goolKeeper = df.filter((col("club_position") == "GK") & (col("potential") > 80) & (col("age") < 30) &(col('goalkeeping_reflexes') > 80) & (col("height_cm") > 195))
goolKeeper.show() """

' goolKeeper = df.filter((col("club_position") == "GK") & (col("potential") > 80) & (col("age") < 30) &(col(\'goalkeeping_reflexes\') > 80) & (col("height_cm") > 195))\ngoolKeeper.show() '

In [ ]:
""" defenderCentralBack = df.filter((col("club_position") == "cb")& (col("potential") > 85) &(col("fifa_version") == 21)  )
defenderCentralBack.show() """

' defenderCentralBack = df.filter((col("club_position") == "cb")& (col("potential") > 85) &(col("fifa_version") == 21)  )\ndefenderCentralBack.show() '

In [ ]:
spark.stop()